<a href="https://colab.research.google.com/github/A01793509/Equipo38/blob/main/MNA_IAyAA_semana_10_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

*   Alberto Jose Garcia Porras (A01793509)
*   Carlos Julio León Caicedo (A01793947)
*   Luis Fernando Ríos Piedra (A00453954)
*   Marco Antonio Vázquez Morales (A01793704)


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [6]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests, zipfile
import seaborn as sns
from io import BytesIO

from sklearn.decomposition import TruncatedSVD,randomized_svd

#**Ejercicio-1.** 

1. Descarga los archivos y genera la matriz de utilidad en la cual los renglones son los usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluaciones generales dadas por los usuarios (rating). En particular indica el tamaño de dicha matriz y despliega 
sus primeros 10 renglones. 

In [7]:
#Extracción de rating_final.csv y geoplaces2.csv desde el sitio fuente

path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00232/RCdata.zip'
base_datos = path.split('/')[-1]
req_path = requests.get(path)

zipfile.ZipFile(BytesIO(req_path.content)).extractall('/content/sample_data/RCdata')
df_rt = pd.read_csv("/content/sample_data/RCdata/rating_final.csv",sep=',')

path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00232/RCdata.zip'
base_datos = path.split('/')[-1]
req_path = requests.get(path)

zipfile.ZipFile(BytesIO(req_path.content)).extractall('/content/sample_data/RCdata')
df_g = pd.read_csv("/content/sample_data/RCdata/geoplaces2.csv",sep=',',encoding='latin-1')


In [8]:
#Generar matriz relacional y asignar index con userID
df_rel=pd.merge(df_rt,df_g,how='inner')
df_rel=df_rel.set_index('userID')


In [9]:
df_rel.shape

(1161, 24)

In [10]:
#Generar matriz de utilidad e impresión de 10 renglones
UtMx = df_rel.pivot_table(values='rating', index='userID', columns='name', fill_value=0)

UtMx.head(10)

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [11]:
#Desplegar tamaño de la matriz
#Tenemos 138 usuarios y 129 restaurantes
UtMx.shape 

(138, 129)

#**Ejercicio-2.**

In [12]:
print("***************************************************")
print('Total de elementos de la matriz de utilidad: %d' % (UtMx.size))
print("***************************************************")
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(UtMx)))
print("*************************************************")
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(UtMx) / UtMx.size))
print("*************************************************")

***************************************************
Total de elementos de la matriz de utilidad: 17802
***************************************************
Total de elementos diferentes de cero: 907
*************************************************
Porcentaje de elementos diferentes de cero: 5.1%
*************************************************


In [13]:
X = UtMx.T
X.shape

(129, 138)

In [37]:
Componentes=137
SVDCompleto = TruncatedSVD(n_components=Componentes, random_state=1)  
SVDCompleto.fit(X)

#num_sv = 197

print("******************************************")
print('El total de valores singulares es de: %d' % (SVDCompleto.singular_values_).shape[0])
print("******************************************")

******************************************
El total de valores singulares es de: 129
******************************************


#**Ejercicio-3.**



---

📈 Ejecutaremos el modelo de recomendación utilizando la variable **rating**.


---



In [15]:
Matriz_Resultante = SVDCompleto.fit_transform(X)
corrMx = np.corrcoef(Matriz_Resultante, )

Restaurante_Evaluado = "Subway"

names = UtMx.columns
names_list = list(names)
id_megusta = names_list.index(Restaurante_Evaluado)

corr_recomienda = corrMx[id_megusta]

In [16]:
Lista_Restaurantes=list(names[(corr_recomienda > .23) & (corr_recomienda < 1)])
i=0
print("*********************************************************************************")
print("Tomando como referencia el restaurante Subway las 10 mejores recomendaciones son:")
print("*********************************************************************************")
for Restaurante in Lista_Restaurantes:
  i=i+1
  Id_Lista = names_list.index(Restaurante)
  print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],4),')')

*********************************************************************************
Tomando como referencia el restaurante Subway las 10 mejores recomendaciones son:
*********************************************************************************
1 - Restaurante Chilis Cuernavaca  (Coeficiente Pearson = 0.3011 )
2 - Restaurante Log Yin  (Coeficiente Pearson = 0.2346 )
3 - Restaurante Mariscos Tia Licha  (Coeficiente Pearson = 0.3882 )
4 - Restaurante Mikasa  (Coeficiente Pearson = 0.301 )
5 - Restaurante Restaurant Teely  (Coeficiente Pearson = 0.6095 )
6 - Restaurante Restaurant and Bar and Clothesline Carlos N Charlies  (Coeficiente Pearson = 0.4255 )
7 - Restaurante Restaurant los Pinos  (Coeficiente Pearson = 0.2851 )
8 - Restaurante Rincon del Bife  (Coeficiente Pearson = 0.4298 )
9 - Restaurante Sanborns Casa Piedra  (Coeficiente Pearson = 0.4172 )
10 - Restaurante Vips  (Coeficiente Pearson = 0.5063 )


Como se puede observar, los 10 restaurantes mantienen una asociación (relación) con nuestro restaurante de referencia subway, dos restaurantes estan más correlacionados con subway y son Restaurant Teely y Vips. El resto de los restaurantes estan menos correlacionados. 

#**Ejercicio-4.**

In [17]:
Componentes=12
SVD = TruncatedSVD(n_components=Componentes, random_state=1)  
SVD.fit(X)

print("*****************************************************")
print('El total de valores singulares encontrados es de: %d' % (SVD.singular_values_).shape[0])

Matriz_Resultante = SVD.fit_transform(X)

corrMx = np.corrcoef(Matriz_Resultante, )
names = UtMx.columns
names_list = list(names)
id_megusta = names_list.index(Restaurante_Evaluado)
corr_recomienda = corrMx[id_megusta]

Lista_Restaurantes_RedDim=list(names[(corr_recomienda > .9985) & (corr_recomienda < 1)])
i=0
print("*********************************************************************************************************************")
print("Reduciendo la dimensionalidad y manteniendo como referencia el restaurante Subway las 10 mejores recomendaciones son:")
print("*********************************************************************************************************************")
for Restaurante in Lista_Restaurantes_RedDim:
  i=i+1
  Id_Lista = names_list.index(Restaurante)
  print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],4),')')
 


*****************************************************
El total de valores singulares encontrados es de: 12
*********************************************************************************************************************
Reduciendo la dimensionalidad y manteniendo como referencia el restaurante Subway las 10 mejores recomendaciones son:
*********************************************************************************************************************
1 - Restaurante Log Yin  (Coeficiente Pearson = 0.9993 )
2 - Restaurante Mariscos Tia Licha  (Coeficiente Pearson = 0.9987 )
3 - Restaurante Mikasa  (Coeficiente Pearson = 0.9992 )
4 - Restaurante Paniroles  (Coeficiente Pearson = 0.9987 )
5 - Restaurante Restaurant Teely  (Coeficiente Pearson = 0.9996 )
6 - Restaurante Restaurant and Bar and Clothesline Carlos N Charlies  (Coeficiente Pearson = 0.9996 )
7 - Restaurante Restaurant los Pinos  (Coeficiente Pearson = 0.9996 )
8 - Restaurante Rincon del Bife  (Coeficiente Pearson = 0.999

In [18]:
import functools
print("*****************************************************************************")
print("Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:")
print("*****************************************************************************")
print("Se encontraron",len(set(Lista_Restaurantes_RedDim).intersection(Lista_Restaurantes)), "restaurantes iguales.")
print("")
print("************************************************************************************")
print("Los siguientes restaurantes difieren en la recomendación al ajustar dimensionalidad:")
print("************************************************************************************")
[x for x in Lista_Restaurantes + Lista_Restaurantes_RedDim if x not in Lista_Restaurantes or x not in Lista_Restaurantes_RedDim]

*****************************************************************************
Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:
*****************************************************************************
Se encontraron 9 restaurantes iguales.

************************************************************************************
Los siguientes restaurantes difieren en la recomendación al ajustar dimensionalidad:
************************************************************************************


['Chilis Cuernavaca', 'Paniroles']

In [19]:
print("*****************************************************************************")
print('Cantidad de información simplificada con los primeros %d vectores singulares:' % Componentes)
print("*****************************************************************************")
print('%.1f%%' %  (100 * (1- (SVDCompleto.singular_values_[0:Componentes]).sum() / (SVDCompleto.singular_values_).sum())))

*****************************************************************************
Cantidad de información simplificada con los primeros 12 vectores singulares:
*****************************************************************************
72.1%


#**Ejercicio-5.**

In [20]:
Componentes=137
SVD = TruncatedSVD(n_components=Componentes, random_state=1)  
SVD.fit(X)

Matriz_Resultante = SVD.fit_transform(X)
corrMx = np.corrcoef(Matriz_Resultante, )

Restaurante_Evaluado = "tacos de barbacoa enfrente del Tec"

names_list = list(names)
id_megusta = names_list.index(Restaurante_Evaluado)

corr_recomienda = corrMx[id_megusta]

Lista_Restaurantes=list(names[(corr_recomienda > .17) & (corr_recomienda < 1)])
i=0
print("*************************************************************************************************************")
print("Tomando como referencia el restaurante 'tacos de barbacoa enfrente del Tec' las 10 mejores recomendaciones son:")
print("*************************************************************************************************************")
for Restaurante in Lista_Restaurantes:
  i=i+1
  if i<11:
    Id_Lista = names_list.index(Restaurante)
    print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],4),')')

*************************************************************************************************************
Tomando como referencia el restaurante 'tacos de barbacoa enfrente del Tec' las 10 mejores recomendaciones son:
*************************************************************************************************************
1 - Restaurante Carreton de Flautas y Migadas  (Coeficiente Pearson = 0.4078 )
2 - Restaurante Hamburguesas La perica  (Coeficiente Pearson = 0.1911 )
3 - Restaurante Little Cesarz  (Coeficiente Pearson = 0.2585 )
4 - Restaurante Taqueria EL amigo   (Coeficiente Pearson = 0.4372 )
5 - Restaurante carnitas mata calle Emilio Portes Gil  (Coeficiente Pearson = 0.1709 )
6 - Restaurante carnitas_mata  (Coeficiente Pearson = 0.1709 )
7 - Restaurante little pizza Emilio Portes Gil  (Coeficiente Pearson = 0.6538 )
8 - Restaurante palomo tec  (Coeficiente Pearson = 0.1952 )
9 - Restaurante puesto de gorditas  (Coeficiente Pearson = 0.4128 )
10 - Restaurante tacos abi  

In [21]:
Componentes=10
SVD = TruncatedSVD(n_components=Componentes, random_state=1)  
SVD.fit(X)

print("****************************************************")
print('El total de valores singulares encontrados es de: %d' % (SVD.singular_values_).shape[0])

Matriz_Resultante = SVD.fit_transform(X)

corrMx = np.corrcoef(Matriz_Resultante, )

corr_recomienda = corrMx[id_megusta]

Lista_Restaurantes_RedDim=list(names[(corr_recomienda > .998) & (corr_recomienda < 1)])
i=0
print("**************************************************************************************************************************************************")
print("Reduciendo la dimensionalidad y manteniendo como referencia el restaurante 'tacos de barbacoa enfrente del Tec' las 10 mejores recomendaciones son:")
print("**************************************************************************************************************************************************")
for Restaurante in Lista_Restaurantes_RedDim:
  i=i+1
  Id_Lista = names_list.index(Restaurante)
  print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],4),')')

****************************************************
El total de valores singulares encontrados es de: 10
**************************************************************************************************************************************************
Reduciendo la dimensionalidad y manteniendo como referencia el restaurante 'tacos de barbacoa enfrente del Tec' las 10 mejores recomendaciones son:
**************************************************************************************************************************************************
1 - Restaurante Hamburguesas La perica  (Coeficiente Pearson = 0.9983 )
2 - Restaurante Little Cesarz  (Coeficiente Pearson = 0.9982 )
3 - Restaurante Pollo_Frito_Buenos_Aires  (Coeficiente Pearson = 0.9995 )
4 - Restaurante Taqueria EL amigo   (Coeficiente Pearson = 0.9999 )
5 - Restaurante carnitas_mata  (Coeficiente Pearson = 0.9994 )
6 - Restaurante little pizza Emilio Portes Gil  (Coeficiente Pearson = 0.9997 )
7 - Restaurante palomo tec  (Coe

In [22]:
import functools
print("*****************************************************************************")
print("Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:")
print("*****************************************************************************")
print("Se encontraron",len(set(Lista_Restaurantes_RedDim).intersection(Lista_Restaurantes)), "restaurantes iguales.")
print("")
print("************************************************************************************")
print("Los siguientes restaurantes difieren en la recomendación al ajustar dimensionalidad:")
print("************************************************************************************")
[x for x in Lista_Restaurantes + Lista_Restaurantes_RedDim if x not in Lista_Restaurantes or x not in Lista_Restaurantes_RedDim]

*****************************************************************************
Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:
*****************************************************************************
Se encontraron 9 restaurantes iguales.

************************************************************************************
Los siguientes restaurantes difieren en la recomendación al ajustar dimensionalidad:
************************************************************************************


['Carreton de Flautas y Migadas',
 'carnitas mata calle Emilio Portes Gil',
 'Pollo_Frito_Buenos_Aires']

In [23]:
print("*****************************************************************************")
print('Cantidad de información simplificada con los primeros %d vectores singulares:' % Componentes)
print("*****************************************************************************")
print('%.1f%%' %  (100 * (1- (SVDCompleto.singular_values_[0:Componentes]).sum() / (SVDCompleto.singular_values_).sum())))

*****************************************************************************
Cantidad de información simplificada con los primeros 10 vectores singulares:
*****************************************************************************
75.6%




---

✍ **Conclusiones:**


---

Tomando como referencia el restaurante tacos de barbacoa enfrente del Tec y con un número de componentes de 137, observamos que "tacos abi" y "little pizza Emilio Portes Gil" son los restaurantes que estan más correlacionados con nuestro restaruante de referencia, todos los demás tienen una correlación menos a 0.5.  

Si reducimos la dimensionalidad a 10 componentes, lo que nos permitió reducir en 75.6% la información, con la nueva dimensionalidad tenemos 9 restaurantes que estan bastante correlacionados con "tacos de barbacoa en frente del Tec", cabe destacar que dentro de ellos "tacos abi" tiene una correlación perfecta, lo que puede ser indicativo de que da lo mismo comer en cualquiera de los dos lugares, sin embargo, **es recomendable escluirla del análisis**. (este punto en el notebook del profesor dice que se tiene que excluir pero es el que mejor empareja con nuestra referencia.  

Por otra parte, los restaurantes que no estaríamos recomendando, serían: 

*   Carreton de Flautas y Migadas
*   carnitas mata calle Emilio Portes Gil
*   Pollo_Frito_Buenos_Aires


#**Ejercicio-6.**



---

📈 Ejecutaremos el modelo de recomendación utilizando la variable **food_rating**.


---



In [24]:
#Generar matriz relacional y asignar index con userID
df_rel=pd.merge(df_rt,df_g,how='inner')
df_rel=df_rel.set_index('userID')
df_rel.shape

#Generar matriz de utilidad e impresión de 10 renglones
UtMx = df_rel.pivot_table(values='food_rating', index='userID', columns='name', fill_value=0)

UtMx.head(10)
UtMx.shape 

(138, 129)

In [25]:
print("***************************************************")
print('Total de elementos de la matriz de utilidad: %d' % (UtMx.size))
print("***************************************************")
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(UtMx)))
print("*************************************************")
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(UtMx) / UtMx.size))
print("*************************************************")

X = UtMx.T
X.shape

Componentes=137
SVD = TruncatedSVD(n_components=Componentes, random_state=1)  
SVD.fit(X)
print('El total de valores singulares es de: %d' % (SVD.singular_values_).shape[0])
print("******************************************")

***************************************************
Total de elementos de la matriz de utilidad: 17802
***************************************************
Total de elementos diferentes de cero: 895
*************************************************
Porcentaje de elementos diferentes de cero: 5.0%
*************************************************
El total de valores singulares es de: 129
******************************************


In [26]:
Matriz_Resultante = SVD.fit_transform(X)
corrMx = np.corrcoef(Matriz_Resultante, )

Restaurante_Evaluado = "Subway"

names = UtMx.columns
names_list = list(names)
id_megusta = names_list.index(Restaurante_Evaluado)

corr_recomienda = corrMx[id_megusta]

Lista_Restaurantes=list(names[(corr_recomienda > .232) & (corr_recomienda < 1)])
i=0
print("*********************************************************************************")
print("Tomando como referencia el restaurante Subway las 10 mejores recomendaciones son:")
print("*********************************************************************************")
for Restaurante in Lista_Restaurantes:
  i=i+1
  Id_Lista = names_list.index(Restaurante)
  print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],4),')')

*********************************************************************************
Tomando como referencia el restaurante Subway las 10 mejores recomendaciones son:
*********************************************************************************
1 - Restaurante Kiku Cuernavaca  (Coeficiente Pearson = 0.2504 )
2 - Restaurante Mariscos Tia Licha  (Coeficiente Pearson = 0.4152 )
3 - Restaurante Paniroles  (Coeficiente Pearson = 0.3121 )
4 - Restaurante Restaurant Familiar El Chino  (Coeficiente Pearson = 0.2327 )
5 - Restaurante Restaurant Las Mananitas  (Coeficiente Pearson = 0.3317 )
6 - Restaurante Restaurant Teely  (Coeficiente Pearson = 0.8164 )
7 - Restaurante Restaurant and Bar and Clothesline Carlos N Charlies  (Coeficiente Pearson = 0.4689 )
8 - Restaurante Rincon del Bife  (Coeficiente Pearson = 0.3818 )
9 - Restaurante Sanborns Casa Piedra  (Coeficiente Pearson = 0.3657 )
10 - Restaurante Vips  (Coeficiente Pearson = 0.4421 )


In [41]:
Componentes=7
SVD = TruncatedSVD(n_components=Componentes, random_state=1)  
SVD.fit(X)

print("****************************************************")
print('El total de valores singulares encontrados es de: %d' % (SVD.singular_values_).shape[0])

Matriz_Resultante = SVD.fit_transform(X)

corrMx = np.corrcoef(Matriz_Resultante, )
corr_recomienda = corrMx[id_megusta]

Lista_Restaurantes_RedDim=list(names[(corr_recomienda > .9999875) & (corr_recomienda < 1)])
i=0
print("*********************************************************************************************************************")
print("Reduciendo la dimensionalidad y manteniendo como referencia el restaurante Subway las 10 mejores recomendaciones son:")
print("*********************************************************************************************************************")
for Restaurante in Lista_Restaurantes_RedDim:
  i=i+1
  Id_Lista = names_list.index(Restaurante)
  print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],6),')')
  
print("************************************************************************************")
print("Los siguientes restaurantes difieren en la recomendación al ajustar dimensionalidad:")
print("************************************************************************************")
[x for x in Lista_Restaurantes + Lista_Restaurantes_RedDim if x not in Lista_Restaurantes or x not in Lista_Restaurantes_RedDim]

****************************************************
El total de valores singulares encontrados es de: 7
*********************************************************************************************************************
Reduciendo la dimensionalidad y manteniendo como referencia el restaurante Subway las 10 mejores recomendaciones son:
*********************************************************************************************************************
1 - Restaurante El Oceano Dorado  (Coeficiente Pearson = 0.999989 )
2 - Restaurante Mariscos Tia Licha  (Coeficiente Pearson = 0.999993 )
3 - Restaurante McDonalds Centro  (Coeficiente Pearson = 0.999989 )
4 - Restaurante Paniroles  (Coeficiente Pearson = 0.999998 )
5 - Restaurante Restaurant Las Mananitas  (Coeficiente Pearson = 0.999992 )
6 - Restaurante Restaurant Teely  (Coeficiente Pearson = 0.999994 )
7 - Restaurante Restaurant and Bar and Clothesline Carlos N Charlies  (Coeficiente Pearson = 0.999996 )
8 - Restaurante Rincon del

['Chilis Cuernavaca',
 'Giovannis',
 'Kiku Cuernavaca',
 'Restaurant Familiar El Chino',
 'El Oceano Dorado',
 'Mariscos Tia Licha',
 'McDonalds Centro']

In [28]:
print("*****************************************************************************")
print("Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:")
print("*****************************************************************************")
print("Se encontraron",len(set(Lista_Restaurantes_RedDim).intersection(Lista_Restaurantes)), "restaurantes iguales.")
print("")

print("*****************************************************************************")
print('Cantidad de información simplificada con los primeros %d vectores singulares:' % Componentes)
print("*****************************************************************************")
print('%.1f%%' %  (100 * (1- (SVDCompleto.singular_values_[0:Componentes]).sum() / (SVDCompleto.singular_values_).sum())))

*****************************************************************************
Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:
*****************************************************************************
Se encontraron 8 restaurantes iguales.

*****************************************************************************
Cantidad de información simplificada con los primeros 7 vectores singulares:
*****************************************************************************
81.3%




---

✍ **Conclusiones:** 


---

Sin reducir dimensionalidad el restaurante que esta más correlacionado con subway es el "restaurante Teely", el resto mantiene una correlación baja. 

Reduciendo el número de componentes a 7, y en 81.3% la información, tenemos que el máximo de coincidencias encontrada fue de 8.


#**Ejercicio-7.**



---

📈 Ejecutaremos el modelo de recomendación utilizando la variable **service_rating**.


---



In [29]:
#Generar matriz relacional y asignar index con userID
df_rel=pd.merge(df_rt,df_g,how='inner')
df_rel=df_rel.set_index('userID')
df_rel.shape

#Generar matriz de utilidad e impresión de 10 renglones
UtMx = df_rel.pivot_table(values='service_rating', index='userID', columns='name', fill_value=0)

UtMx.head(10)
UtMx.shape 

(138, 129)

In [30]:
print("***************************************************")
print('Total de elementos de la matriz de utilidad: %d' % (UtMx.size))
print("***************************************************")
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(UtMx)))
print("*************************************************")
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(UtMx) / UtMx.size))
print("*************************************************")
X = UtMx.T
X.shape

Componentes=137
SVD = TruncatedSVD(n_components=Componentes, random_state=1)  
SVD.fit(X)

print('El total de valores singulares es de: %d' % (SVD.singular_values_).shape[0])

***************************************************
Total de elementos de la matriz de utilidad: 17802
***************************************************
Total de elementos diferentes de cero: 846
*************************************************
Porcentaje de elementos diferentes de cero: 4.8%
*************************************************
El total de valores singulares es de: 129


In [31]:
Matriz_Resultante = SVD.fit_transform(X)
corrMx = np.corrcoef(Matriz_Resultante, )

Restaurante_Evaluado = "Subway"

names = UtMx.columns
names_list = list(names)
id_megusta = names_list.index(Restaurante_Evaluado)

corr_recomienda = corrMx[id_megusta]

Lista_Restaurantes=list(names[(corr_recomienda > .17) & (corr_recomienda < 1)])
i=0
print("*********************************************************************************")
print("Tomando como referencia el restaurante Subway las 10 mejores recomendaciones son:")
print("*********************************************************************************")
for Restaurante in Lista_Restaurantes:
  i=i+1
  Id_Lista = names_list.index(Restaurante)
  print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],4),')')


*********************************************************************************
Tomando como referencia el restaurante Subway las 10 mejores recomendaciones son:
*********************************************************************************
1 - Restaurante Chilis Cuernavaca  (Coeficiente Pearson = 0.2307 )
2 - Restaurante Giovannis  (Coeficiente Pearson = 0.1928 )
3 - Restaurante Kiku Cuernavaca  (Coeficiente Pearson = 0.1787 )
4 - Restaurante Paniroles  (Coeficiente Pearson = 0.3659 )
5 - Restaurante Restaurant Familiar El Chino  (Coeficiente Pearson = 0.1829 )
6 - Restaurante Restaurant Las Mananitas  (Coeficiente Pearson = 0.2263 )
7 - Restaurante Restaurant Teely  (Coeficiente Pearson = 0.773 )
8 - Restaurante Restaurant and Bar and Clothesline Carlos N Charlies  (Coeficiente Pearson = 0.6002 )
9 - Restaurante Rincon del Bife  (Coeficiente Pearson = 0.5187 )
10 - Restaurante Vips  (Coeficiente Pearson = 0.4212 )


In [42]:
Componentes=15
SVD = TruncatedSVD(n_components=Componentes, random_state=1)  
SVD.fit(X)

print("****************************************************")
print('El total de valores singulares encontrados es de: %d' % (SVD.singular_values_).shape[0])

Matriz_Resultante = SVD.fit_transform(X)

corrMx = np.corrcoef(Matriz_Resultante, )
corr_recomienda = corrMx[id_megusta]

Lista_Restaurantes_RedDim=list(names[(corr_recomienda > .81) & (corr_recomienda < 1)])
i=0
print("*********************************************************************************************************************")
print("Reduciendo la dimensionalidad y manteniendo como referencia el restaurante Subway las 10 mejores recomendaciones son:")
print("*********************************************************************************************************************")
for Restaurante in Lista_Restaurantes_RedDim:
  i=i+1
  Id_Lista = names_list.index(Restaurante)
  print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],4),')')

print("************************************************************************************")
print("Los siguientes restaurantes difieren en la recomendación al ajustar dimensionalidad:")
print("************************************************************************************")
[x for x in Lista_Restaurantes + Lista_Restaurantes_RedDim if x not in Lista_Restaurantes or x not in Lista_Restaurantes_RedDim]

****************************************************
El total de valores singulares encontrados es de: 15
*********************************************************************************************************************
Reduciendo la dimensionalidad y manteniendo como referencia el restaurante Subway las 10 mejores recomendaciones son:
*********************************************************************************************************************
1 - Restaurante Chilis Cuernavaca  (Coeficiente Pearson = 0.924 )
2 - Restaurante El Oceano Dorado  (Coeficiente Pearson = 0.8136 )
3 - Restaurante El cotorreo  (Coeficiente Pearson = 0.9007 )
4 - Restaurante Paniroles  (Coeficiente Pearson = 0.9028 )
5 - Restaurante Restaurant Teely  (Coeficiente Pearson = 0.8904 )
6 - Restaurante Restaurant and Bar and Clothesline Carlos N Charlies  (Coeficiente Pearson = 0.899 )
7 - Restaurante Restaurant los Pinos  (Coeficiente Pearson = 0.8383 )
8 - Restaurante Rincon del Bife  (Coeficiente Pears

['Giovannis',
 'Kiku Cuernavaca',
 'Restaurant Familiar El Chino',
 'Restaurant Las Mananitas',
 'El Oceano Dorado',
 'El cotorreo',
 'Restaurant los Pinos',
 'Sanborns Casa Piedra']

In [33]:
print("*****************************************************************************")
print("Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:")
print("*****************************************************************************")
print("Se encontraron",len(set(Lista_Restaurantes_RedDim).intersection(Lista_Restaurantes)), "restaurantes iguales.")
print("")
print("*****************************************************************************")
print('Cantidad de información simplificada con los primeros %d vectores singulares:' % Componentes)
print("*****************************************************************************")
print('%.1f%%' %  (100 * (1- (SVDCompleto.singular_values_[0:Componentes]).sum() / (SVDCompleto.singular_values_).sum())))

*****************************************************************************
Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:
*****************************************************************************
Se encontraron 6 restaurantes iguales.

*****************************************************************************
Cantidad de información simplificada con los primeros 15 vectores singulares:
*****************************************************************************
67.1%




---

✍ **Conclusiones:** 


---

Sin reducir dimensionalidad tres restaurantes mantienen una correlación alta con respecto a Subway, en primer término "restaurante Teely", seguido de "Restaurant and Bar and Clothesline Carlos N Charlies" y finalmente "Rincón del Bife".

Reduciendo la información en un 67.1%, es decir, a solo 15 componentes, observamos que el máximo de coincidencias encontrada fue de 6 restaurantes.


#**Ejercicio-8.**



---

✍ **Conclusiones Finales:**


---

Del análisis de los tres modelos llegamos a las siguientes conclusiones:



1.   En cada uno de los modelos sin reducción de dimensionalidad se puede observar una correlación de los restaurantes con nuestro restaurante de referencia. 
2.   La reducción de dimensionalidad ayuda a encontrar mayores similitudes entre los restaurantes y nuestra referencia, lo que nos permite tener mejores recomendaciones. 
3.   Realizar el análisis desde el rating promedio global del restaurante incrementa las coincidencias de restaurantes, lo que nos indica que existe información importante que no es considerada para la recomendación.
4.   El especificar el análisis especificando que rating usar, como por ejemplo "food rating" o "service rating", reduce las coincidencias y aumenta los restaurantes no recomendados.
5.  Reducir el número de componentes o valores singulares, ayuda a reducir la cantidad de información lo que nos permite tener un mejor sistema de recomendación.







###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**